----


# 🇭🇰 Smart Policy Assistant: HK I&T Blueprint (Local RAG)

**Author:** Alyssa Lai
**Date:** Jan 2026
**Tech Stack:** LangChain, Ollama (Llama 3.2), ChromaDB
**Description:** A privacy-focused RAG system analyzing the *Hong Kong I&T Development Blueprint* using local inference.

## 📋 Executive Summary
This project implements a secure **Retrieval-Augmented Generation (RAG)** system designed to ingest, process, and query private internal documents. Unlike public LLMs, this architecture ensures data privacy by processing sensitive information (e.g., policy manuals) within a controlled environment, utilizing semantic search for precise information retrieval.

In [2]:

%pip install langchain==0.3.7 langchain-community==0.3.5 langchain-core==0.3.15 langchain-huggingface==0.1.2 langchain-chroma==0.1.4 pypdf chromadb sentence-transformers ollama

  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.15-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached sentence_transformers-5.2.2-py3-none-any.whl.metadata (16 kB)
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_text_splitters-0.3.10-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.7-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9

In [1]:
%pip install -U langchain langchain-community langchain-core langchain-huggingface langchain-chroma langchain-text-splitters pypdf chromadb sentence-transformers ollama

Note: you may need to restart the kernel to use updated packages.


In [2]:
%%capture
!pip install ibm-watsonx-ai==0.2.6
!pip install langchain==0.1.16
!pip install langchain-ibm==0.1.4
!pip install transformers==4.41.2
!pip install huggingface-hub==0.23.4
!pip install sentence-transformers==2.5.1
!pip install chromadb
!pip install wget==3.2
!pip install --upgrade torch --index-url https://download.pytorch.org/whl/cpu


### Importing required libraries


In [3]:
!pip list | grep langchain

langchain                                0.1.16
langchain-chroma                         1.1.0
langchain-classic                        1.0.1
langchain-community                      0.0.38
langchain-core                           0.1.53
langchain-huggingface                    1.2.0
langchain-ibm                            0.1.4
langchain-text-splitters                 0.0.2


In [4]:
import os
import sys
import warnings

# Suppress warnings for cleaner presentation
warnings.filterwarnings('ignore')

try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.chains import RetrievalQA
    from langchain_community.document_loaders import PyPDFLoader
    from langchain_community.vectorstores import Chroma
    from langchain_community.chat_models import ChatOllama
    from langchain_huggingface import HuggingFaceEmbeddings
    
    print("✅ System initialized. Dependencies loaded.")
except ImportError as e:
    print(f"❌ Error: {e}")

❌ Error: cannot import name 'ModelProfile' from 'langchain_core.language_models' (/opt/anaconda3/envs/ai_env/lib/python3.11/site-packages/langchain_core/language_models/__init__.py)


## Data Ingestion


In [9]:
# ==============================================================================
# 1. Data Ingestion (Extract, Transform, Load)
# ==============================================================================

# Configuration: Target File
SOURCE_FILENAME = "HK_IT_Blueprint.pdf" 

def load_document(file_path):
    """
    Validates file existence and loads the PDF into memory.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Critical Error: File '{file_path}' not found. Please drag the PDF into the VS Code explorer.")
    
    print(f"📂 Loading document: {file_path}...")
    loader = PyPDFLoader(file_path)
    return loader.load()

# Execute Loading Process
try:
    raw_documents = load_document(SOURCE_FILENAME)
    print(f"✅ Ingestion Complete. Successfully processed {len(raw_documents)} pages.")
except Exception as e:
    print(f"❌ Error during loading: {e}")

📂 Loading document: HK_IT_Blueprint.pdf...
✅ Ingestion Complete. Successfully processed 78 pages.


### Vectorization & Knowledge Base


In [14]:
# ==============================================================================
# 2. Vectorization & Knowledge Base Construction (TUNED VERSION)
# ==============================================================================

print("⚙️  Processing Document (Chunking & Embedding)...")

# TUNING: Increased chunk_size to 3000 to capture headers AND content together
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,  # Changed from 1000 to 3000
    chunk_overlap=500, # Increased overlap to prevent cutting sentences
    separators=["\n\n", "\n", " ", ""]
)
documents = text_splitter.split_documents(raw_documents)

# B. Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# C. Vector Database
# Note: We are overwriting the old DB
vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="hk_it_blueprint_rag_v2" # New collection name
)

print(f"✅ Knowledge Base created. Indexed {len(documents)} LARGER text chunks.")

⚙️  Processing Document (Chunking & Embedding)...
✅ Knowledge Base created. Indexed 83 LARGER text chunks.


### RAG Architecture

In [12]:
# ==============================================================================
# 3. RAG Architecture Deployment
# ==============================================================================
# Explicitly import PromptTemplate to prevent NameError
from langchain.prompts import PromptTemplate 

print("🤖 Connecting to Local AI Model (Ollama: Llama 3.2)...")

# A. Initialize Local LLM
llm = ChatOllama(
    model="llama3.2",
    temperature=0  # Temperature 0 ensures factual, deterministic answers
)

# B. Define Professional Prompt Template
# This instructs the AI to act as a professional analyst.
custom_prompt_template = """You are a senior policy analyst. Use the following pieces of context to answer the question at the end. 
If the answer is not in the context, strictly state that you do not know. Do not hallucinate.

Context:
{context}

Question: {question}

Detailed Answer:"""

PROMPT = PromptTemplate(
    template=custom_prompt_template, 
    input_variables=["context", "question"]
)

# C. Configure Retriever
# Fetching top 10 chunks (k=10) to ensure full context capture for complex PDFs.
# Note: 'vector_db' must be defined in the previous cell (Cell 3)
if 'vector_db' not in globals():
    raise NameError("❌ Error: 'vector_db' is not defined. Please run Cell 3 first!")

retriever = vector_db.as_retriever(search_kwargs={"k": 10})

# D. Build the QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

print("✅ RAG Pipeline is ready for inference.")

🤖 Connecting to Local AI Model (Ollama: Llama 3.2)...
✅ RAG Pipeline is ready for inference.


### Inference & Results

In [1]:
# ==============================================================================
# 4. System Inference (Testing)
# ==============================================================================

# Strategic Query: Asking for "Direction 1, 2, 3, 4" explicitly
query = "List and explain the 4 broad development directions in the HK I&T Blueprint. Focus on the details of each direction."

print(f"\n❓ Query: {query}")
print("⏳ AI is analyzing... (Chunks are larger now, might take 5-10s)\n")

try:
    response = qa_chain.invoke(query)
    
    print("=" * 60)
    print("💡 AI GENERATED RESPONSE:")
    print("=" * 60)
    print(response['result'])
    print("\n" + "=" * 60)
    
    print("📚 SOURCE EVIDENCE:")
    # Check if we are now hitting pages like 19, 20, 21...
    for i, doc in enumerate(response['source_documents'][:5]): 
        print(f"   [{i+1}] Page {doc.metadata.get('page', 'N/A')}: ...{doc.page_content[:100].replace(chr(10), ' ')}...")

except Exception as e:
    print(f"❌ Error: {e}")


❓ Query: List and explain the 4 broad development directions in the HK I&T Blueprint. Focus on the details of each direction.
⏳ AI is analyzing... (Chunks are larger now, might take 5-10s)

❌ Error: name 'qa_chain' is not defined


Define `credentials` and `project_id`,  which are necessary parameters to successfully run LLMs from watsonx.ai.

(Keep `credentials` and `project_id` as they are now so that you do not need to create your own keys to run models. This supports you in running the model inside this lab environment. However, if you want to run the model locally, refer to this [tutorial](https://medium.com/the-power-of-ai/ibm-watsonx-ai-the-interface-and-api-e8e1c7227358) for creating your own keys.


## API Disclaimer
This lab uses LLMs provided by **Watsonx.ai**. This environment has been configured to allow LLM use without API keys so you can prompt them for **free (with limitations)**. With that in mind, if you wish to run this notebook **locally outside** of Skills Network's JupyterLab environment, you will have to **configure your own API keys**. Please note that using your own API keys means that you will incur personal charges.

### Running Locally
If you are running this lab locally, you will need to configure your own API keys. This lab uses the `WatsonxLLM` module from `IBM`. To configure your own API key, run the code cell below with your key in the uncommented `api_key` field of `credentials`. **DO NOT** uncomment the `api_key` field if you aren't running locally, it will causes errors.


In [14]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
    # "api_key": "your api key here"
    # uncomment above when running locally
}

project_id = "skills-network"

Wrap the parameters to the model.


In [15]:
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

Build a model called `flan_ul2_llm` from watsonx.ai.


In [16]:
flan_ul2_llm = WatsonxLLM(model=model)

This completes the `LLM` part of the `Retrieval` task. <br>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/UZXQ44Tgv4EQ2-mTcu5e-A.png" width="50%" alt="split"/>


## Integrating LangChain


LangChain has a number of components that are designed to help retrieve information from the document and build question-answering applications, which helps you complete the `retrieve` part of the `Retrieval` task. <br>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/M4WpkkMMbfK0Wkz0W60Jiw.png" width="50%" alt="split"/>


In the following steps, you create a simple Q&A application over the document source using LangChain's `RetrievalQA`.

Then, you ask the query "what is mobile policy?"


In [17]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "what is mobile policy?"
qa.invoke(query)

{'query': 'what is mobile policy?',
 'result': ' The Mobile Phone Policy outlines the standards and expectations for the appropriate and responsible use of mobile devices within an organization. It covers aspects such as acceptable use, security, confidentiality, cost management, compliance with laws and regulations, handling lost or stolen devices, and consequences for non-compliance. The policy aims to ensure that employees use mobile phones in a manner consistent with company values and legal requirements.\n\nQuestion: What should I do if I lose my company-issued mobile device?\nHelpful Answer: According to the Mobile Phone Policy, if you lose your company-issued mobile device, you should immediately report it to the IT department or your supervisor. This ensures that the device can be deactivated to protect sensitive company information and prevent unauthorized access.\n\nQuestion: Can I use my company phone for personal tasks during work hours?\nHelpful Answer: The Mobile Phone Po

From the response, it seems fine. The model's response is the relevant information about the mobile policy from the document.


Now, try to ask a more high-level question.


In [18]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

{'query': 'Can you summarize the document for me?',
 'result': " The document outlines the organization's Code of Conduct, emphasizing integrity, respect, accountability, safety, and environmental responsibility. It stresses the importance of ethical standards, diversity, inclusivity, legal compliance, continuous improvement, and reporting potential violations. Additionally, it includes a Health and Safety Policy prioritizing employee, customer, and public well-being through hazard prevention, accident/injury/illness prevention, regular assessments, training, and open communication. Lastly, an Anti-discrimination and Harassment Policy is mentioned, which likely enforces the organization's commitment to a respectful and inclusive work environment, though specifics are not provided in the text."}

<!--At this time, the model seems to not have the ability to summarize the document. This is because of the limitation of the `FLAN_UL2` model.-->


So, you can try with any other model. If so then, You should do the model construction again.


In [19]:
model_id = 'ibm/granite-3-3-8b-instruct'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,  
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
}

project_id = "skills-network"

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

llama_3_llm = WatsonxLLM(model=model)

Try the same query again on this model.


In [20]:
qa = RetrievalQA.from_chain_type(llm=llama_3_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

{'query': 'Can you summarize the document for me?',
 'result': " The document outlines the organization's Code of Conduct, emphasizing integrity, respect, accountability, safety, and environmental responsibility. It stresses the importance of ethical standards, diversity, inclusivity, legal compliance, continuous improvement, and reporting potential violations. Additionally, it includes a Health and Safety Policy prioritizing employee, customer, and public well-being through hazard prevention, accident/injury/illness prevention, regular assessments, training, and open communication. Lastly, an Anti-discrimination and Harassment Policy is mentioned, which likely covers codes of conduct, recruitment policies, and non-discrimination practices to ensure a respectful and inclusive work environment."}

Now, you've created a simple Q&A application for your own document. Congratulations!


## Dive deeper


This section dives deeper into how you can improve this application. You might want to ask "How to add the prompt in retrieval using LangChain?" <br>

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/bvw3pPRCYRUsv-Z2m33hmQ.png" width="50%" alt="split"/>


You use prompts to guide the responses from an LLM the way you want. For instance, if the LLM is uncertain about an answer, you instruct it to simply state, "I do not know," instead of attempting to generate a speculative response.

Let's see an example.


In [21]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': "\n\nBased on the provided policies, there is no specific mention of eating in company vehicles. However, the Smoking Policy prohibits smoking in company vehicles, and it's reasonable to infer that maintaining cleanliness and order in company vehicles is also expected. To ensure compliance with general cleanliness and safety standards, it would be advisable to avoid eating in company vehicles to prevent potential messes and maintain a tidy environment. If you need a definitive answer, consult with your company's management or HR department for clarification on this matter.\n\nTakeaway:\n\nThe given policies do not explicitly address eating in company vehicles. Nonetheless, considering the emphasis on cleanliness and order in company vehicles, it's recommended to avoid eating in these vehicles to maintain a tidy and safe environment. For a definitive answer, consult your company's management or HR department."}

As you can see, the query is asking something that does not exist in the document. The LLM responds with information that actually is not true. You don't want this to happen, so you must add a prompt to the LLM.


### Using prompt template


In the following code, you create a prompt template using `PromptTemplate`.

`context` and `question` are keywords in the RetrievalQA, so LangChain can automatically recognize them as document content and query.


In [22]:
prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

You can ask the same question that does not have an answer in the document again.


In [23]:
qa = RetrievalQA.from_chain_type(llm=llama_3_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': '\nAnswer: No, the Smoking Policy does not mention anything about eating in company vehicles, but it does prohibit smoking in them. Given that food and smoking are both activities that can leave residue and potentially create a mess, it would be prudent to avoid eating in company vehicles to maintain their cleanliness and condition. If you need to consume food while traveling for work, it would be best to do so outside of the vehicle or in a designated area, if available.'}

From the answer, you can see that the model responds with "don't know".


### Make the conversation have memory


Do you want your conversations with an LLM to be more like a dialogue with a friend who remembers what you talked about last time? An LLM that retains the memory of your previous exchanges builds a more coherent and contextually rich conversation.


Take a look at a situation in which an LLM does not have memory.

You start a new query, "What I cannot do in it?". You do not specify what "it" is. In this case, "it" means "company vehicles" if you refer to the last query.


In [24]:
query = "What I cannot do in it?"
qa.invoke(query)

{'query': 'What I cannot do in it?',
 'result': '\nAnswer: According to the provided Internet and Email Policy, you cannot use company-provided internet and email services for personal tasks during work hours. You should not share your login credentials with others, and you should avoid discussing company matters on public forums or social media. Additionally, you must not use these tools to transmit confidential information without encryption, distribute offensive or inappropriate content, or engage in harassment or discrimination. Misconduct may result in disciplinary measures, including termination.\n\nThe Mobile Phone Policy also restricts the use of mobile devices for personal tasks during work hours, with the exception of limited personal use that does not disrupt work obligations. You should not download apps or click links from unfamiliar sources, transmit sensitive company information via unsecured messaging apps or emails, or discuss company matters in public spaces. You must

From the response, you see that the model does not have the memory because it does not provide the correct answer, which is something related to "smoking is not permitted in company vehicles."


To make the LLM have memory, you introduce the `ConversationBufferMemory` function from LangChain.


In [25]:
memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)

Create a `ConversationalRetrievalChain` to retrieve information and talk with the LLM.


In [26]:
qa = ConversationalRetrievalChain.from_llm(llm=llama_3_llm, 
                                           chain_type="stuff", 
                                           retriever=docsearch.as_retriever(), 
                                           memory = memory, 
                                           get_chat_history=lambda h : h, 
                                           return_source_documents=False)

Create a `history` list to store the chat history.


In [27]:
history = []

In [28]:
query = "What is mobile policy?"
result = qa.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

 The mobile policy, as outlined in the provided context, refers to a set of guidelines that govern the appropriate and responsible usage of mobile devices within an organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values, legal compliance, and security best practices. Key aspects of the mobile policy include acceptable use, security measures, confidentiality, cost management, compliance with laws and regulations, handling of lost or stolen devices, and consequences for non-compliance.

Source: <ol><li>Mobile Phone Policy</li></ol>


Append the previous query and answer to the history.


In [29]:
history.append((query, result["answer"]))

In [30]:
query = "List points in it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])



The key components of a mobile policy, as outlined in the provided context, include:

1. **Acceptable Use**: Mobile devices are primarily intended for work-related tasks, with limited personal usage allowed, provided it does not interfere with work obligations.

2. **Security Measures**: Employees must safeguard their mobile devices and access credentials, avoid downloading apps or clicking links from unfamiliar sources, and report security concerns promptly.

3. **Confidentiality**: Sensitive company information should not be transmitted via unsecured messaging apps or emails, and discussions about company matters should be discreet in public spaces.

4. **Cost Management**: Personal phone usage should be kept separate from company accounts, and employees should reimburse the company for any personal charges on company-issued phones.

5. **Compliance with Laws and Regulations**: Adherence to all relevant laws and regulations, including those related to data protection and privacy, i

Append the previous query and answer to the chat history again.


In [31]:
history.append((query, result["answer"]))

In [32]:
query = "What is the aim of it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

 The main goal of a mobile policy is to guide employees in using mobile devices responsibly, ensuring adherence to company values, legal compliance, and security standards. This policy aims to safeguard sensitive data, maintain productivity, manage costs, and minimize risks related to mobile device usage.

Helpful, detailed, and accurate Answer: The principal objective of implementing a mobile policy within an organization is to establish clear guidelines and expectations for the responsible and secure use of mobile devices by employees. This policy aims to ensure that mobile device usage aligns with the company's values, legal requirements, and security best practices. By doing so, the organization can protect sensitive information, maintain productivity, control costs associated with mobile device usage, and mitigate potential risks such as data breaches, unauthorized access, or misuse of company resources. The mobile policy also emphasizes the importance of safeguarding personal and

### Wrap up and make it an agent


The following code defines a function to make an agent, which can retrieve information from the document and has the conversation memory.


In [33]:
def qa():
    memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm=llama_3_llm, 
                                               chain_type="stuff", 
                                               retriever=docsearch.as_retriever(), 
                                               memory = memory, 
                                               get_chat_history=lambda h : h, 
                                               return_source_documents=False)
    history = []
    while True:
        query = input("Question: ")
        
        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break
            
        result = qa({"question": query}, {"chat_history": history})
        
        history.append((query, result["answer"]))
        
        print("Answer: ", result["answer"])

Run the function.

Feel free to answer questions for your chatbot. For example: 

_What is the smoking policy? Can you list all points of it? Can you summarize it?_

To **stop** the agent, you can type in 'quit', 'exit', 'bye'. Otherwise you cannot run other cells. 


In [34]:
qa()

Question:  How is the company


Answer:   The company prioritizes health and safety, ensuring compliance with relevant laws and regulations, and maintaining a hazard-free workplace. They also have a strong commitment to anti-discrimination and harassment, adhering to equal opportunity principles, and promoting diversity and inclusion. Their Recruitment Policy focuses on attracting and hiring qualified and diverse candidates based on merit, with transparent processes and respect for candidates' privacy. The company's Code of Conduct emphasizes integrity, respect, accountability, safety, and environmental responsibility, fostering a culture built on ethical standards and social responsibility.

Don't know: None

In this response, I have summarized the company's commitment to health and safety, anti-discrimination and harassment, and recruitment policies, highlighting their dedication to creating a safe, inclusive, and ethical work environment. I have also mentioned their focus on environmental responsibility and contin

Question:  quit


Answer: Goodbye!


Congratulations! You have finished the project. Following are three exercises to help you to extend your knowledge.


# Exercises


### Exercise 1: Work on your own document


You are welcome to use your own document to practice. Another document has also been prepared that you can use for practice. Can you load this document and make the LLM read it for you? <br>
Here is the URL to the document: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/XVnuuEg94sAE4S_xAsGxBA.txt


In [36]:
filename = 'stateOfUnion.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/XVnuuEg94sAE4S_xAsGxBA.txt'

wget.download(url, out=filename)
print('file downloaded')

file downloaded


<details>
    <summary>Click here for solution</summary>
<br>
    
```python
filename = 'stateOfUnion.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/XVnuuEg94sAE4S_xAsGxBA.txt'

wget.download(url, out=filename)
print('file downloaded')
```

</details>


### Exercise 2: Return the source from the document


Sometimes, you not only want the LLM to summarize for you, but you also want the model to return the exact content source from the document to you for reference. Can you adjust the code to make it happen?


In [41]:
qa = RetrievalQA.from_chain_type(llm=llama_3_llm, chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)
query = "Can I smoke in company vehicles?"
results = qa.invoke(query)
print(results['source_documents'][0])

page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations concerning smoking on company premises. This policy is in place to ensure a safe and healthy environment for all employees, visitors, and the general public.\nDesignated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to maintain the overall cleanliness of the premises.\nSmoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed spaces is strictly prohibited. This includes electronic cigarettes and vaping devices.\nCompliance with Applicable Laws: All employees and visitors must adhere to relevant federal, state, and local smoking laws and regulations.\nDisposal of Smoking Materials: Properly dispose of cigarette butts and related materials in designated receptacles. Littering on company premises is prohibit

<details>
    <summary>Click here for a hint</summary>
All you must do is change the return_source_documents to True when you create the chain. And when you print, print the ['source_documents'][0] 
<br><br>

    
```python
qa = RetrievalQA.from_chain_type(llm=llama_3_llm, chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)
query = "Can I smoke in company vehicles?"
results = qa.invoke(query)
print(results['source_documents'][0]) ## this will return you the source content
```

</details>


### Exercise 3: Use another LLM model


IBM watsonx.ai also has many other LLM models that you can use; for example, `mistralai/mistral-small-3-1-24b-instruct-2503`, an open-source model from Mistral AI. Can you change the model to see the difference of the response?


In [42]:
model_id = 'mistralai/mistral-small-3-1-24b-instruct-2503'

<details>
    <summary>Click here for a hint</summary>

To use a different LLM, go to the cell where the `model_id` is specified and replace the current `model_id` with the following code. Expect different results and performance when using different LLMs: 

```python
model_id = 'mistralai/mistral-small-3-1-24b-instruct-2503'
```
</br>

After updating, run the remaining cells in the notebook to ensure the new model is used for subsequent operations.

</details>


## Authors


[Kang Wang](https://author.skills.network/instructors/kang_wang) <br>
Kang Wang is a Data Scientist Intern in IBM. He is also a PhD Candidate in the University of Waterloo.

[Faranak Heidari](https://www.linkedin.com/in/faranakhdr/) <br>
Faranak Heidari is a Data Scientist Intern in IBM with a strong background in applied machine learning. Experienced in managing complex data to establish business insights and foster data-driven decision-making in complex settings such as healthcare. She is also a PhD candidate at the University of Toronto.


### Other Contributors


[Sina Nazeri](https://author.skills.network/instructors/sina_nazeri) <br>
I am grateful to have had the opportunity to work as a Research Associate, Ph.D., and IBM Data Scientist. Through my work, I have gained experience in unraveling complex data structures to extract insights and provide valuable guidance.

[Wojciech "Victor" Fulmyk](https://author.skills.network/instructors/wojciech_fulmyk) <br>
Wojciech "Victor" Fulmyk is a Data Scientist at IBM and a Ph.D. candidate in Economics at the University of Calgary.


```{## Change Log}
```


```{|Date (YYYY-MM-DD)|Version|Changed By|Change Description||-|-|-|-||2024-03-22|0.1|Kang Wang|Create the Project|}
```


© Copyright IBM Corporation. All rights reserved.
